
first block initializes a dataset you can use for training

In [ ]:
import pandas as pd

#this is the dataset from the example in class, use it for testing
data = [['Susan','F','Blue','O-','excellent',75,'N','N'],['Jim','M','Red','O+','good',65,'N','N'],
        ['Joe','M','Red','AB-','fair',64,'N','Y'],['Jane','F','Green','A+','poor',83,'Y','Y'],
        ['Sam','M','Blue','A-','good',71,'N','N'],['Michelle','F','Blue','O-','good',90,'N','N']]


df = pd.DataFrame( data, columns = ['Name', 'Gender', 'Favorite Color', 'Blood Type', 'General Health', 'Test1', 'Cough', 'High Blood Pressure'] ).drop( 'Name', axis=1 )

#this is for the function where the column is tested for ordinal or not and then used when making it numeric
#I'll give you that function because of this
ordinal_order = ['poor', 'fair', 'good', 'excellent']
df['General Health'] = pd.Categorical(df['General Health'], categories=ordinal_order, ordered=True)

more training data if you want it. I provide 2 sets for you and a function that can create more


In [ ]:
import numpy as np

# Create a 10x10 DataFrame with columns of different types

#here's more training data. It's just a function that creates training sets with a set seed and size
def createDataset( seed, rows ):
  np.random.seed(seed)
  data1 = {
      'Gender': np.random.choice(['Male', 'Female'], size=rows),
      'Region': np.random.choice(['North', 'South', 'East', 'West'], size=rows),
      'Is_Eligible': np.random.choice(['Y', 'N'], size=rows),
      'Pet_Type': np.random.choice(['Dog', 'Cat', 'Bird'], size=rows),
      'Favorite_Color': np.random.choice(['Red', 'Blue', 'Green'], size=rows),
      'Education_Level': pd.Categorical(np.random.choice(['High School', 'College', 'Graduate School'], size=rows, p=[0.2, 0.5, 0.3]), categories=['High School', 'College', 'Graduate School'], ordered=True),
      'Age': np.random.randint(18, 60, size=rows),
      'Income': np.random.uniform(20000, 80000, size=rows),
      'Height': np.random.normal(160, 10, size=rows),
      'Has_Pet_Insurance': np.random.choice(['Y', 'N'], size=rows)
  }

  return pd.DataFrame(data1)

#2 for you to try out since I promised I'd provide 3 training sets
train2 = createDataset( 14, 10 )
train3 = createDataset( 42, 15 )


Uncomment a line and run to avoid having to change anything else below when testing with a different dataset

In [ ]:
df = train2
#df = train3

This block returns a dictionary containing every column and its datatype (symmetric binary, asymmetric binary, nominal, ordinal, or numeric)

In [ ]:
def getDataType(column):

    #Easiest test of all, juts test if its dtype is numeric
    if pd.api.types.is_numeric_dtype(column):
        return 'Numeric'

    countValues = column.unique()

    #In all of the data I give you to test and use to grade this assignment, Asymmetric Binary data will be Y/N
    if len(countValues) == 2 and set(countValues) == {'Y', 'N'}:
        return 'Asymmetric Binary'

    #Since Asymmetric Binary is Y/N, Symmetric Binary will have 2 unique values that aren't Y/N
    if len(countValues) == 2 and set(countValues) != {'Y', 'N'}:
        return 'Symmetric Binary'

    #This uses the code from the previous block to test for ordinal
    if isinstance(column.dtype, pd.CategoricalDtype) and column.dtype.ordered:
        return 'Ordinal'

    #If it's categorical and not ordinal, it's nominal
    if pd.api.types.is_categorical_dtype(column) or pd.api.types.is_object_dtype(column):
        return 'Nominal'

#This is just a dictionary mapping each column and its datatype. Might be useful
dataTypes = {}
for column in df.columns:
  dataTypes[column] = getDataType(df[column])

In [ ]:
dataTypes

{'Gender': 'Symmetric Binary',
 'Favorite Color': 'Nominal',
 'Blood Type': 'Nominal',
 'General Health': 'Ordinal',
 'Test1': 'Numeric',
 'Cough': 'Asymmetric Binary',
 'High Blood Pressure': 'Asymmetric Binary'}

compute the distances


In [ ]:
#You should pass a dataframe of just the symmetric binary columns here and return a matrix of the distance
def symmetric_binary_dist( data ):


  df = pd.DataFrame(data, columns=['Name','Gender', 'Favorite Color', 'Blood Type', 'General Health', 'Test1', 'Cough', 'High Blood Pressure']).drop('Name', axis=1)
  ordinal_order = ['poor', 'fair', 'good', 'excellent']
  df['General Health'] = pd.Categorical(df['General Health'], categories=ordinal_order, ordered=True)

  for column_name, column_data in df.items():
    if getDataType(column_data) != 'Symmetric Binary':
      df = df.drop(column_name, axis=1)

  num_col = len(df.columns)
  num_rows = len(df)
  col_name = df.columns
  col_name = list(col_name)

  dist = [[0 for j in range(num_rows)] for i in range(num_rows)]

  for i in range(0,num_col):
    for j in range(0,num_rows):
      for z in range(0,num_rows):
        if df.at[j,col_name[i]]==df.at[z,col_name[i]]:
          dist[j][z] = 0
        else:
          dist[j][z] = 1

  return dist
symmetric_binary_dist(data)

[[0, 1, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1],
 [1, 0, 0, 1, 0, 1],
 [0, 1, 1, 0, 1, 0],
 [1, 0, 0, 1, 0, 1],
 [0, 1, 1, 0, 1, 0]]

In [ ]:
#You should pass a dataframe of just the nominal columns here and return a matrix of the distance
def nominal_dist( data ):
  dist = [[]]

  df = pd.DataFrame(data, columns=['Name','Gender', 'Favorite Color', 'Blood Type', 'General Health', 'Test1', 'Cough', 'High Blood Pressure']).drop('Name', axis=1)
  ordinal_order = ['poor', 'fair', 'good', 'excellent']
  df['General Health'] = pd.Categorical(df['General Health'], categories=ordinal_order, ordered=True)

  for column_name, column_data in df.iteritems():
    if getDataType(column_data) != 'Nominal':
      df = df.drop(column_name, axis=1)

  num_col = len(df.columns)
  num_rows = len(df)
  col_name = df.columns
  col_name = list(col_name)

  dist = [[0 for j in range(num_rows)] for i in range(num_rows)]

  for i in range(0,num_col):
    for j in range(0,num_rows):
      for z in range(0,num_rows):
        if df.at[j,col_name[i]]==df.at[z,col_name[i]]:
          dist[j][z] += 0
        else:
          dist[j][z] += .5

  return dist

<ipython-input-101-876a8bc908de>:9: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in df.iteritems():


[[0, 1.0, 1.0, 1.0, 0.5, 0],
 [1.0, 0, 0.5, 1.0, 1.0, 1.0],
 [1.0, 0.5, 0, 1.0, 1.0, 1.0],
 [1.0, 1.0, 1.0, 0, 1.0, 1.0],
 [0.5, 1.0, 1.0, 1.0, 0, 0.5],
 [0, 1.0, 1.0, 1.0, 0.5, 0]]

In [ ]:
#You should pass a dataframe of just the ordinal columns here and return a matrix of the distance
#Suggestion: maybe just convert to numeric and run the numeric_dist method
def ordinal_dist( data ):
  dist = [[]]

  df = pd.DataFrame(data, columns=['Name','Gender', 'Favorite Color', 'Blood Type', 'General Health', 'Test1', 'Cough', 'High Blood Pressure']).drop('Name', axis=1)
  ordinal_order = ['poor', 'fair', 'good', 'excellent']
  df['General Health'] = pd.Categorical(df['General Health'], categories=ordinal_order, ordered=True)

  for column_name, column_data in df.iteritems():
    if getDataType(column_data) != 'Ordinal':
      df = df.drop(column_name, axis=1)

  replace_d = {'excellent':1, 'good':2/3, 'fair':1/3, 'poor':0}

  df['General Health'] = df['General Health'].replace(replace_d)


  num_col = len(df.columns)
  num_rows = len(df)
  col_name = df.columns
  col_name = list(col_name)

  dist = [[0 for j in range(num_rows)] for i in range(num_rows)]

  for i in range(0,num_col):
    for j in range(0,num_rows):
      for z in range(0,num_rows):
        dist[j][z] = round(abs(df.at[j, col_name[i]]-df.at[z, col_name[i]]), 3)

  return dist

<ipython-input-99-2f34e9a5a25d>:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in df.iteritems():


[[0.0, 0.333, 0.667, 1.0, 0.333, 0.333],
 [0.333, 0.0, 0.333, 0.667, 0.0, 0.0],
 [0.667, 0.333, 0.0, 0.333, 0.333, 0.333],
 [1.0, 0.667, 0.333, 0.0, 0.667, 0.667],
 [0.333, 0.0, 0.333, 0.667, 0.0, 0.0],
 [0.333, 0.0, 0.333, 0.667, 0.0, 0.0]]

In [ ]:
#You should pass a dataframe of just the numeric columns here and return a matrix of the distance
def numeric_dist( data ):
  dist = [[]]

  df = pd.DataFrame(data, columns=['Name','Gender', 'Favorite Color', 'Blood Type', 'General Health', 'Test1', 'Cough', 'High Blood Pressure']).drop('Name', axis=1)
  ordinal_order = ['poor', 'fair', 'good', 'excellent']
  df['General Health'] = pd.Categorical(df['General Health'], categories=ordinal_order, ordered=True)

  for column_name, column_data in df.iteritems():
    if getDataType(column_data) != 'Numeric':
      df = df.drop(column_name, axis=1)

  num_col = len(df.columns)
  num_rows = len(df)
  col_name = df.columns
  col_name = list(col_name)
  replace = {}

  for z in range(0, num_col):
    min = df[col_name[z]].min()
    max = df[col_name[z]].max()
    df[col_name[z]] = df[col_name[z]].replace(replace)
    for f in range(0,num_rows):
      num = df.at[f, col_name[z]]
      replace[num] = (num-min)/(max-min)
  df[col_name[z]] = df[col_name[z]].replace(replace)


  dist = [[0 for j in range(num_rows)] for i in range(num_rows)]

  for i in range(0,num_col):
    for j in range(0,num_rows):
      for z in range(0,num_rows):
        dist[j][z] = round(abs(df.at[j, col_name[i]]-df.at[z, col_name[i]]), 3)


  return dist

<ipython-input-100-8c92dfa1c3f4>:9: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in df.iteritems():


[[0.0, 0.385, 0.423, 0.308, 0.154, 0.577],
 [0.385, 0.0, 0.038, 0.692, 0.231, 0.962],
 [0.423, 0.038, 0.0, 0.731, 0.269, 1.0],
 [0.308, 0.692, 0.731, 0.0, 0.462, 0.269],
 [0.154, 0.231, 0.269, 0.462, 0.0, 0.731],
 [0.577, 0.962, 1.0, 0.269, 0.731, 0.0]]

In [ ]:
#You should pass a dataframe of just the asymmetric binary columns here and return a matrix of the distance
def asymmetric_binary_dist( data ):
  dist = [[]]

  df = pd.DataFrame(data, columns=['Name','Gender', 'Favorite Color', 'Blood Type', 'General Health', 'Test1', 'Cough', 'High Blood Pressure']).drop('Name', axis=1)
  ordinal_order = ['poor', 'fair', 'good', 'excellent']
  df['General Health'] = pd.Categorical(df['General Health'], categories=ordinal_order, ordered=True)

  for column_name, column_data in df.iteritems():
    if getDataType(column_data) != 'Asymmetric Binary':
      df = df.drop(column_name, axis=1)

  num_col = len(df.columns)
  num_rows = len(df)
  col_name = df.columns
  col_name = list(col_name)

  dist = [[0 for j in range(num_rows)] for i in range(num_rows)]

  for i in range(0,num_col):
    for j in range(0,num_rows):
      for z in range(0,num_rows):
        if df.at[j,col_name[i]]== 'N' and df.at[z,col_name[i]] == 'N':
          dist[j][z] += 0
        elif (df.at[j,col_name[i]]== 'Y' and df.at[z,col_name[i]] == 'N') or (df.at[j,col_name[i]]== 'N' and df.at[z,col_name[i]] == 'Y'):
          dist[j][z] += .5
  return dist

asymmetric_binary_dist( data )

<ipython-input-128-5cf501f0353d>:9: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in df.iteritems():


[[0, 0, 0.5, 1.0, 0, 0],
 [0, 0, 0.5, 1.0, 0, 0],
 [0.5, 0.5, 0, 0.5, 0.5, 0.5],
 [1.0, 1.0, 0.5, 0, 1.0, 1.0],
 [0, 0, 0.5, 1.0, 0, 0],
 [0, 0, 0.5, 1.0, 0, 0]]

In [ ]:
#This should probably call every distance method and aggregate the results.
#Don't forget to make sure each individual distance matrix is weighted by the number of columns of that data type
def get_dist( data ):

  df = pd.DataFrame(data, columns=['Name','Gender', 'Favorite Color', 'Blood Type', 'General Health', 'Test1', 'Cough', 'High Blood Pressure']).drop('Name', axis=1)
  ordinal_order = ['poor', 'fair', 'good', 'excellent']
  df['General Health'] = pd.Categorical(df['General Health'], categories=ordinal_order, ordered=True)

  num_col = len(df.columns)
  num_rows = len(df)
  col_name = df.columns
  col_name = list(col_name)

  dist_sym = [[0 for j in range(num_rows)] for i in range(num_rows)]
  dist_nom = [[0 for j in range(num_rows)] for i in range(num_rows)]
  dist_ord = [[0 for j in range(num_rows)] for i in range(num_rows)]
  dist_numeric = [[0 for j in range(num_rows)] for i in range(num_rows)]
  dist_asy = [[0 for j in range(num_rows)] for i in range(num_rows)]


  dist_sym = symmetric_binary_dist(data)
  dist_nom = nominal_dist(data)
  dist_ord = ordinal_dist(data)
  dist_numeric = numeric_dist(data)
  dist_asy = asymmetric_binary_dist(data)

  for i in range(num_rows):
    for y in range(num_rows):
      dist_sym[i][y] = round(dist_sym[i][y]*1/7,3)
      dist_nom[i][y] = round(dist_nom[i][y]*2/7,3)
      dist_ord[i][y] = round(dist_ord[i][y]*1/7,3)
      dist_numeric[i][y] = round(dist_numeric[i][y]*1/7,3)
      dist_asy[i][y] = round(dist_asy[i][y]*2/7,3)

  return "Symmetric Binary", dist_sym,"Nominal", dist_nom, "Ordinal", dist_ord, "Numeric", dist_numeric, "Asymetric Binary", dist_asy

get_dist(data)

<ipython-input-101-876a8bc908de>:9: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in df.iteritems():
<ipython-input-99-2f34e9a5a25d>:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in df.iteritems():
<ipython-input-100-8c92dfa1c3f4>:9: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in df.iteritems():
<ipython-input-128-5cf501f0353d>:9: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column_name, column_data in df.iteritems():


('Symmetric Binary',
 [[0.0, 0.143, 0.143, 0.0, 0.143, 0.0],
  [0.143, 0.0, 0.0, 0.143, 0.0, 0.143],
  [0.143, 0.0, 0.0, 0.143, 0.0, 0.143],
  [0.0, 0.143, 0.143, 0.0, 0.143, 0.0],
  [0.143, 0.0, 0.0, 0.143, 0.0, 0.143],
  [0.0, 0.143, 0.143, 0.0, 0.143, 0.0]],
 'Nominal',
 [[0.0, 0.286, 0.286, 0.286, 0.143, 0.0],
  [0.286, 0.0, 0.143, 0.286, 0.286, 0.286],
  [0.286, 0.143, 0.0, 0.286, 0.286, 0.286],
  [0.286, 0.286, 0.286, 0.0, 0.286, 0.286],
  [0.143, 0.286, 0.286, 0.286, 0.0, 0.143],
  [0.0, 0.286, 0.286, 0.286, 0.143, 0.0]],
 'Ordinal',
 [[0.0, 0.048, 0.095, 0.143, 0.048, 0.048],
  [0.048, 0.0, 0.048, 0.095, 0.0, 0.0],
  [0.095, 0.048, 0.0, 0.048, 0.048, 0.048],
  [0.143, 0.095, 0.048, 0.0, 0.095, 0.095],
  [0.048, 0.0, 0.048, 0.095, 0.0, 0.0],
  [0.048, 0.0, 0.048, 0.095, 0.0, 0.0]],
 'Numeric',
 [[0.0, 0.055, 0.06, 0.044, 0.022, 0.082],
  [0.055, 0.0, 0.005, 0.099, 0.033, 0.137],
  [0.06, 0.005, 0.0, 0.104, 0.038, 0.143],
  [0.044, 0.099, 0.104, 0.0, 0.066, 0.038],
  [0.022, 0.03